In [1]:
import pandas as pd
import sqlite3


In [2]:
conn = sqlite3.connect("pos.db")


In [3]:
query = """
SELECT
  m.item_id,
  m.item_name,
  m.category,
  m.price,
  m.prep_time_minutes,
  s.units_sold_last_month,
  s.profit_margin_percent,
  ROUND(
    (m.price * s.units_sold_last_month)
    * (s.profit_margin_percent / 100.0), 2
  ) AS profit
FROM restaurant_menu m
JOIN menu_sales s
ON m.item_id = s.item_id
"""
df = pd.read_sql(query, conn)



In [4]:
df.head()

,item_id,item_name,category,price,prep_time_minutes,units_sold_last_month,profit_margin_percent,profit
0,101,Garlic Knots,Appetizers,8.99,5,159,46,657.53
1,102,Loaded Fries,Appetizers,12.50,8,490,38,2327.50
2,103,Bruschetta,Appetizers,10.50,6,46,43,207.69
3,104,Stuffed Mushrooms,Appetizers,11.00,7,62,64,436.48
4,105,Buffalo Wings,Appetizers,13.50,9,361,75,3655.13


In [5]:
documents = []

for _, row in df.iterrows():
    text = (
        f"Menu item '{row.item_name}' in category '{row.category}' "
        f"is priced at {row.price}. "
        f"It sold {row.units_sold_last_month} units last month, "
        f"with a profit margin of {row.profit_margin_percent}%. "
        f"It generated a total profit of {row.profit}, "
        f"which is {'very high' if row.profit > df['profit'].quantile(0.75) else 'moderate or low'} "
        f"compared to other menu items. "
        f"Average preparation time is {row.prep_time_minutes} minutes."
    )
    documents.append({
        "content": text,
        "metadata": {
            "item_id": row.item_id,
            "category": row.category,
            "source": "pos_sql"
        }
    })

documents[:2]


[{'content': "Menu item 'Garlic Knots' in category 'Appetizers' is priced at 8.99. It sold 159 units last month, with a profit margin of 46%. It generated a total profit of 657.53, which is moderate or low compared to other menu items. Average preparation time is 5 minutes.",
  'metadata': {'item_id': 101, 'category': 'Appetizers', 'source': 'pos_sql'}},
 {'content': "Menu item 'Loaded Fries' in category 'Appetizers' is priced at 12.5. It sold 490 units last month, with a profit margin of 38%. It generated a total profit of 2327.5, which is very high compared to other menu items. Average preparation time is 8 minutes.",
  'metadata': {'item_id': 102, 'category': 'Appetizers', 'source': 'pos_sql'}}]

In [6]:
for doc in documents[:3]:
    print(doc["content"])
    print("---")


Menu item 'Garlic Knots' in category 'Appetizers' is priced at 8.99. It sold 159 units last month, with a profit margin of 46%. It generated a total profit of 657.53, which is moderate or low compared to other menu items. Average preparation time is 5 minutes.
---
Menu item 'Loaded Fries' in category 'Appetizers' is priced at 12.5. It sold 490 units last month, with a profit margin of 38%. It generated a total profit of 2327.5, which is very high compared to other menu items. Average preparation time is 8 minutes.
---
Menu item 'Bruschetta' in category 'Appetizers' is priced at 10.5. It sold 46 units last month, with a profit margin of 43%. It generated a total profit of 207.69, which is moderate or low compared to other menu items. Average preparation time is 6 minutes.
---


In [7]:
pop_high = df["units_sold_last_month"].quantile(0.7)
profit_high = df["profit"].quantile(0.7)

df["menu_class"] = df.apply(
    lambda r: (
        "High-High" if r.units_sold_last_month >= pop_high and r.profit >= profit_high else
        "High-Low" if r.units_sold_last_month >= pop_high else
        "Low-High" if r.profit >= profit_high else
        "Low-Low"
    ),
    axis=1
)


for i, row in df.iterrows():
    documents[i]["content"] += f" This item is classified as {row.menu_class}."



In [8]:
import json

with open("rag_documents.json", "w") as f:
    json.dump(documents, f, indent=2)

print("RAG documents saved.")


RAG documents saved.


In [9]:
import json

with open("rag_documents.json") as f:
    rag_docs = json.load(f)

len(rag_docs)


53

In [10]:
from langchain_core.documents import Document


documents = [
    Document(
        page_content=doc["content"],
        metadata=doc["metadata"]
    )
    for doc in rag_docs
]


In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


/var/folders/s1/s94qmf4129q34gqmyptfrk4w0000gn/T/ipykernel_29911/124733854.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [12]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embedding_model)
vectorstore.save_local("faiss_index")


In [13]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})



In [14]:
queries = [
    "Which menu items are most profitable?",
    "Which items sell a lot but have low margins?",
    "Items that take long to prepare"
]

for q in queries:
    print(f"\nQUERY: {q}")
    docs = retriever.invoke(q)
    for d in docs:
        print("-", d.page_content)




QUERY: Which menu items are most profitable?
- Menu item 'Mac and Cheese' in category 'Sides' is priced at 7.0. It sold 167 units last month, with a profit margin of 37%. It generated a total profit of 432.53, which is moderate or low compared to other menu items. Average preparation time is 5 minutes. This item is classified as Low-Low.
- Menu item 'Bruschetta' in category 'Appetizers' is priced at 10.5. It sold 46 units last month, with a profit margin of 43%. It generated a total profit of 207.69, which is moderate or low compared to other menu items. Average preparation time is 6 minutes. This item is classified as Low-Low.
- Menu item 'Nacho Platter' in category 'Appetizers' is priced at 13.0. It sold 430 units last month, with a profit margin of 76%. It generated a total profit of 4248.4, which is very high compared to other menu items. Average preparation time is 9 minutes. This item is classified as High-High.
- Menu item 'Chicken Alfredo' in category 'Main_Courses' is priced 